<table>
    <tr>
        <td><img src="https://s3.amazonaws.com/media-p.slid.es/uploads/1485763/images/9060062/Header.png" width="300"/></td>
        <td>&nbsp;</td>
        <td>
            <h1 style="font-size:200%;color:blue;text-align:center">    <FONT COLOR="blue">  Caso Final  </FONT>         </h1></td>         
        <td>
            <tp><p style="font-size:99%;text-align:center">PLN </p></tp>
            <tp><p style="font-size:115%;text-align:center">Pregrado MACC 2023-2</p></tp>
            <tp><p style="font-size:115%;text-align:center">Prof. Fabián Sánchez</p></tp>
        </td>
    </tr>
</table>

# <FONT SIZE=5 COLOR="green"> **Objetivo del estudio de caso y resultados de aprendizaje** </FONT>

En este estudio de caso se espera que el estudiante compare diversos métodos para resolver problemas de clasificación. Particularmente.

- Cargar y aplicar las técnicas de limpieza para normalizar un texto.

- Aplicar algunas técnicas de machine learning para clasificación

- Aplicar algunas estructuras de redes neuronales (MLP-LSTM) para clasificación.

- Comparar los resultados.

**objetivo**: dado un conjunto de datos de reviews de vinos, considerar las dos variables: *description* y *variety*. Determinar la variedad del vino a partir de descripción del mismo.



# <FONT SIZE=5 COLOR="green"> **Indicaciones** </FONT>

- Para entregar en grupos de máximo tres integrantes.

- Cargar en la plataforma *e-aulas* los archivos: *.ipynb* y *.pdf*

- Fecha de entrega el día **martes 28 de Noviembre 6:00 p.m.**

- Preparar una presentación de los resultados del caso para el día **miércoles 29 de Noviembre de 2023** en la sesión de clase.

- Entrega de notas: viernes **1 de diciembre**



# <FONT SIZE=5 COLOR="bronw"> **Punto 1. Carga y Exploración de los datos** </FONT>

1. Considerar el conjunto de datos ***wine_review***.

2. Realizar una breve descripción de los datos.

3. Realizar una tabla de frecuencias y gráfico de barras de la variable *variety*.

4. Del punto anterior seleccionar el top 10. Es decir, las 10 variedades de vino con más registros.

5. De cada una de las 10 variedades de vino, seleccionar 2000 registros de manera aleatoria. Es decir, la base final quedará constituida por 20.000 registros.

6. Realizar el proceso de normalización (limpieza) de la variable *description*.Tenga en cuenta guardar el pre-procesado en un archivo *pickle*.

7. Hacer la nube de palabras de la variable *description*.

# <FONT SIZE=5 COLOR="bronw"> **Punto 2. Modelos ML and ANN** </FONT>

1. Seleccione dos modelos de machine learning para resolver el problema de clasificación.

2. Aplique MLP (multilayer perceptron) con *Tensorflow-Keras* para generar el modelo.

3. Aplique una estructura de LSTM con *Tensorflow-Keras* para generar el modelo.

4. Evalúe y compare los resultados de los modelos

5. Concluya

# <FONT SIZE=5 COLOR="bronw"> **Punto 3. Topic Modeling** </FONT>

- Utilice el metodo LDA visto en clase para generar clustering a partir de la variable *description*.
- Use la visualización LDA para explorar las palabras más relevantes de los grupos.
- Concluya